In [2]:
import Pkg

#Pkg.add(url="https://github.com/matthiasbaitsch/mmjmesh.git");

using IntervalSets
using MMJMesh.Plots
using MMJMesh.Meshes
using MMJMesh.Utilities
using MMJMesh.Mathematics
using LinearAlgebra
using Symbolics
using DomainSets
using CairoMakie

using GLMakie
using WGLMakie
WGLMakie.activate!()
CairoMakie.activate!()

Definition Variablen

In [3]:
@variables  a,b, h, ν,E ,p;

Einheitsmatrix

In [4]:
I12 = Matrix{Int}(I, 12, 12);

In [5]:
V = [ 0 a a 0; 0 0 b b]

P = mmonomials(2, 3, QHat , (p1, p2) -> p1 + p2 <= 4 && p1 * p2 <4,type = Int)
N = vcat(
    [
        [
            ValueAtLF(p),
            PDerivativeAtLF(p, [1, 0]),
            PDerivativeAtLF(p, [0, 1])
        ]
        for p in eachcol(V)
    ]...
)
M = [n(p) for p in P, n in N]
Minv = simplify.(M \ I12)
H4 = Minv*P
#fplot3d(H4)

12-element Vector{AbstractMapping{StaticArraysCore.SVector{2, <:Real}, Float64}}:
 -2 / ((a^3)*b)x₁³x₂+-2 / (a*(b^3))x₁x₂³+2 / (a^3)x₁³+3 / ((a^2)*b)x₁²x₂+3 / (a*(b^2))x₁x₂²+2 / (b^3)x₂³+-3 / (a^2)x₁²+-1 / (a*b)x₁x₂+-3 / (b^2)x₂²+1
 -1 / ((a^2)*b)x₁³x₂+1 / (a^2)x₁³+2 / (a*b)x₁²x₂+-2 / ax₁²+-1 / bx₁x₂+1x₁
 -1 / (a*(b^2))x₁x₂³+2 / (a*b)x₁x₂²+1 / (b^2)x₂³+-1 / ax₁x₂+-2 / bx₂²+1x₂
 2 / ((a^3)*b)x₁³x₂+2 / (a*(b^3))x₁x₂³+-2 / (a^3)x₁³+-3 / ((a^2)*b)x₁²x₂+-3 / (a*(b^2))x₁x₂²+3 / (a^2)x₁²+1 / (a*b)x₁x₂
 -1 / ((a^2)*b)x₁³x₂+1 / (a^2)x₁³+1 / (a*b)x₁²x₂+-1 / ax₁²
 1 / (a*(b^2))x₁x₂³+-2 / (a*b)x₁x₂²+1 / ax₁x₂
 -2 / ((a^3)*b)x₁³x₂+-2 / (a*(b^3))x₁x₂³+3 / ((a^2)*b)x₁²x₂+3 / (a*(b^2))x₁x₂²+-1 / (a*b)x₁x₂
 1 / ((a^2)*b)x₁³x₂+-1 / (a*b)x₁²x₂
 1 / (a*(b^2))x₁x₂³+-1 / (a*b)x₁x₂²
 2 / ((a^3)*b)x₁³x₂+2 / (a*(b^3))x₁x₂³+-3 / ((a^2)*b)x₁²x₂+-3 / (a*(b^2))x₁x₂²+-2 / (b^3)x₂³+1 / (a*b)x₁x₂+3 / (b^2)x₂²
 1 / ((a^2)*b)x₁³x₂+-2 / (a*b)x₁²x₂+1 / bx₁x₂
 -1 / (a*(b^2))x₁x₂³+1 / (a*b)x₁x₂²+1 / (b^2)x₂³+-1 / bx₂²

In [6]:
L = [1 ν 0; ν 1 0; 0 0 (1-ν)/2];

In [7]:
∂1(u) =  (∂x(∂x(u)))
∂2(u) =  (∂y(∂y(u)))
∂3(u) =  2*(∂x(∂y(u)))
Be1(u) = [∂1(u), ∂2(u), ∂3(u)]
Be2(u) = [∂1(u) + ν * ∂2(u), ν * ∂1(u) + ∂2(u),(1- ν)/2 * ∂3(u)]

ae(u,v) = integrate(Be1(u) ⋅ Be2(v), (0 .. a) , (0 .. b))
Ke = (((E*h^3) / (12*(1-ν^2)))* simplifyx.([ae(n1, n2) for n1 ∈ H4, n2 ∈ H4]))

12×12 Matrix{Num}:
 (E*(20(a^4) + 14(a^2)*(b^2) + 20(b^4) - 4(a^2)*(b^2)*ν)*(h^3)) / (60(a^3)*(b^3)*(1 - (ν^2)))   …   (E*(10(a^2) + b^2 - (b^2)*ν)*(h^3)) / (60a*(b^2)*(1 - (ν^2)))
 (E*(a^2 + 10(b^2) + 4(a^2)*ν)*(h^3)) / (60(a^2)*b*(1 - (ν^2)))                                     0
 (E*(10(a^2) + b^2 + 4(b^2)*ν)*(h^3)) / (60a*(b^2)*(1 - (ν^2)))                                     (E*(10(a^2) - (b^2) + (b^2)*ν)*(h^3)) / (180a*b*(1 - (ν^2)))
 (E*(10(a^4) - 14(a^2)*(b^2) - 20(b^4) + 4(a^2)*(b^2)*ν)*(h^3)) / (60(a^3)*(b^3)*(1 - (ν^2)))       (E*(5(a^2) - (b^2) + (b^2)*ν)*(h^3)) / (60a*(b^2)*(1 - (ν^2)))
 (E*(a^2 + 10(b^2) - (a^2)*ν)*(h^3)) / (60(a^2)*b*(1 - (ν^2)))                                      0
 (E*(5(a^2) - (b^2) - 4(b^2)*ν)*(h^3)) / (60a*(b^2)*(1 - (ν^2)))                                …   (E*(5(a^2) + b^2 - (b^2)*ν)*(h^3)) / (180a*b*(1 - (ν^2)))
 (E*(-10(a^4) + 14(a^2)*(b^2) - 10(b^4) - 4(a^2)*(b^2)*ν)*(h^3)) / (60(a^3)*(b^3)*(1 - (ν^2)))      (E*(-5(a^2) + b^2 + 4(b^2)*ν)*(h^

Output der Elementsteifigkeitsmatrix

In [8]:
KeNew = zeros(12,12)
for i = 1:12
    for j = 1:12
        if KeNew[i,j] == 1
        else 
            KeNew[i,j] = 1 
            print("Ke[",i,",",j,"]")
            for m = 1:12
                for n = 1:12
                    if m==i && j==n 
                    elseif isequal((Ke[i,j]),(Ke[m,n])) == true 
                        KeNew[m,n] = 1.0
                        print(" = ", "Ke[",m,",",n,"]")
                    # elseif isequal(expand(Ke[i,j]),expand(-1*(Ke[m,n]))) == true 
                    #     KeNew[m,n] = 1.0
                    #     print(" = ", "-Ke[",m,",",n,"]")
                    else
                    end
                end
            end
        print(" = ", Ke[i,j])
        println()
        end
    end
end

Ke[1,1] = Ke[4,4] = Ke[7,7] = Ke[10,10] = (E*(20(a^4) + 14(a^2)*(b^2) + 20(b^4) - 4(a^2)*(b^2)*ν)*(h^3)) / (60(a^3)*(b^3)*(1 - (ν^2)))
Ke[1,2] = Ke[2,1] = Ke[10,11] = Ke[11,10] = (E*(a^2 + 10(b^2) + 4(a^2)*ν)*(h^3)) / (60(a^2)*b*(1 - (ν^2)))
Ke[1,3] = Ke[3,1] = Ke[4,6] = Ke[6,4] = (E*(10(a^2) + b^2 + 4(b^2)*ν)*(h^3)) / (60a*(b^2)*(1 - (ν^2)))
Ke[1,4] = Ke[4,1] = Ke[7,10] = Ke[10,7] = (E*(10(a^4) - 14(a^2)*(b^2) - 20(b^4) + 4(a^2)*(b^2)*ν)*(h^3)) / (60(a^3)*(b^3)*(1 - (ν^2)))
Ke[1,5] = Ke[5,1] = Ke[8,10] = Ke[10,8] = (E*(a^2 + 10(b^2) - (a^2)*ν)*(h^3)) / (60(a^2)*b*(1 - (ν^2)))
Ke[1,6] = Ke[3,4] = Ke[4,3] = Ke[6,1] = (E*(5(a^2) - (b^2) - 4(b^2)*ν)*(h^3)) / (60a*(b^2)*(1 - (ν^2)))
Ke[1,7] = Ke[4,10] = Ke[7,1] = Ke[10,4] = (E*(-10(a^4) + 14(a^2)*(b^2) - 10(b^4) - 4(a^2)*(b^2)*ν)*(h^3)) / (60(a^3)*(b^3)*(1 - (ν^2)))
Ke[1,8] = Ke[5,10] = Ke[8,1] = Ke[10,5] = (E*(-(a^2) + 5(b^2) + (a^2)*ν)*(h^3)) / (60(a^2)*b*(1 - (ν^2)))
Ke[1,9] = Ke[4,12] = Ke[9,1] = Ke[12,4] = (E*(5(a^2) - (b^2) + (b^2)*ν

Vergleichswerte aus dem Buch von Klein

In [9]:
beta2 = a/b
gamma = b/a
fKlein = ((E*h^3) / (1440*(1-(ν^2))*a*b));
ke11Klein = print(fKlein * (120 * (beta2^2 + gamma^2) - 24ν +84));
ke12Klein = print(fKlein * (10beta2^2 + (1 + 4ν)) * 12b);
ke13Klein = print(fKlein * -10 * (10gamma^2 +(1 + 4ν)) * 12a);
ke14Klein = print(fKlein * 60 * (gamma^2 - 2beta2^2) + 24ν - 84);

(E*(h^3)*(84 - 24ν + 120((a / b)^2 + (b / a)^2))) / (1440a*b*(1 - (ν^2)))(E*(h^3)*(1 + 4ν + 10((a / b)^2))) / (120a*(1 - (ν^2)))(-E*(h^3)*(1 + 4ν + 10((b / a)^2))) / (12b*(1 - (ν^2)))-84 + (E*(h^3)*(-2((a / b)^2) + (b / a)^2)) / (24a*b*(1 - (ν^2))) + 24ν

In [10]:
isequal((Ke[1,1]),(Ke[4,4]))

true

In [11]:
isequal(print(Ke[1,4]),ke14Klein)

(E*(10(a^4) - 14(a^2)*(b^2) - 20(b^4) + 4(a^2)*(b^2)*ν)*(h^3)) / (60(a^3)*(b^3)*(1 - (ν^2)))

true

In [12]:
fe = p * a * b * 1/3 * [3,b,-a,3,b,a,3,-b,a,3,-b,-a]

12-element Vector{Num}:
             a*b*p
  (1//3)*a*(b^2)*p
 (-1//3)*(a^2)*b*p
             a*b*p
  (1//3)*a*(b^2)*p
  (1//3)*(a^2)*b*p
             a*b*p
 (-1//3)*a*(b^2)*p
  (1//3)*(a^2)*b*p
             a*b*p
 (-1//3)*a*(b^2)*p
 (-1//3)*(a^2)*b*p

In [16]:
f(u) = integrate(u, (0 .. a) , (0 .. b))
FeN =  p * simplifyx.([f(n1) for n1 ∈ H4])

12-element Vector{Num}:
       (1//4)*a*b*p
  (1//24)*(a^2)*b*p
  (1//24)*a*(b^2)*p
       (1//4)*a*b*p
 (-1//24)*(a^2)*b*p
  (1//24)*a*(b^2)*p
       (1//4)*a*b*p
 (-1//24)*(a^2)*b*p
 (-1//24)*a*(b^2)*p
       (1//4)*a*b*p
  (1//24)*(a^2)*b*p
 (-1//24)*a*(b^2)*p